In [1]:
import trimesh
import json
import os
import multiprocessing as mp
from tqdm import tqdm

In [2]:
DATA_ROOT = '/data/lixinag/project/GarmentCode/A_DATA/GarmentCodeData'
SPLIT_FILE_PATH = '/data/lixinag/project/GarmentCode/A_DATA/GarmentCodeData/train_test_split.json'

In [3]:
def transfer_garment_ply_2_garment_obj_with_shift(file_path, 
        scale_factor=0.01, shift_distance=1.16):
    # load the garment ply file
    mesh = trimesh.load_mesh(file_path, process=False)
    mesh_vertices = mesh.vertices.tolist()
    mesh_faces = mesh.faces.tolist()

    # scale the vertices
    mesh_vertices_scaled = [
        [vertex[0] * scale_factor, vertex[1] * scale_factor, vertex[2] * scale_factor]
        for vertex in mesh_vertices
    ]

    # shift the vertices
    mesh_vertices_shifted = [
        [vertex[0], vertex[1] - shift_distance, vertex[2]]
        for vertex in mesh_vertices_scaled
    ]

    # write the obj file
    # obj_file_path = file_path.replace('.ply', '.obj')
    obj_file_path = '/data/lixinag/project/GarmentCode/A_DATA/tmp/' + file_path.split('/')[-1].replace('.ply', '.obj')
    with open(obj_file_path, 'w') as f:
        for vertex in mesh_vertices_shifted:
            f.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        for face in mesh_faces:
            f.write(f"f {face[0] + 1} {face[1] + 1} {face[2] + 1}\n")


In [4]:
# main function
input_file_path_pool = []
with open(SPLIT_FILE_PATH, 'r') as f:
    data = json.load(f)
    for key_name in data.keys():
        for file_path in data[key_name]:
            file_id = file_path.split('/')[-1]
            input_file_path_pool.append(os.path.join(DATA_ROOT, file_path, file_id + '_sim.ply'))

# mp and tqdm
# mp.Pool(processes=int(mp.cpu_count() * 0.8)).map(transfer_garment_ply_2_garment_obj_with_shift, input_file_path_pool)
with mp.Pool(processes=int(mp.cpu_count() * 0.8)) as pool:
    results = list(tqdm(
        pool.imap(transfer_garment_ply_2_garment_obj_with_shift, input_file_path_pool),
        total=len(input_file_path_pool),
        desc="Converting PLY to OBJ"
    ))

Converting PLY to OBJ: 100%|██████████| 115195/115195 [05:28<00:00, 350.38it/s]


In [13]:
[file_path.split('/')[-1] for file_path in input_file_path_pool]
input_file_path_pool[7]

'/data/lixinag/project/GarmentCode/A_DATA/GarmentCodeData/garments_5000_0/default_body/rand_7PYIP67HEM/rand_7PYIP67HEM_sim.ply'

In [12]:
import trimesh
import PIL

path_to_mesh = input_file_path_pool[7]
path_to_texture_img = '/data/lixinag/project/GarmentCode/A_DATA/GarmentCodeData/garments_5000_0/default_body/rand_RVS1QWPXD0/rand_RVS1QWPXD0_texture.png'

mesh = trimesh.load_mesh(path_to_mesh, process=False)
tex_image = PIL.Image.open(path_to_texture_img)
tex = trimesh.visual.TextureVisuals(mesh.visual.uv, image=tex_image)
mesh.visual = tex
mesh.show()